# Segmenting and Clustering Neighborhoods in Toronto - Part 2

Part 1:
Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe. Clean the data.

Part 2:
Collect latitude and longitude for every postal code in the dataframe, and append to the dataframe

Perform imports

In [18]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import geocoder

print('Libraries imported.')

Libraries imported.


Read in wikipedia data into pandas dataframe. This can be done with read_html.

In [60]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
table = table[0]
table.dropna(axis=0, subset=['Neighborhood'], inplace=True)

mysep='\n---------------------------------------------------------------------------------------------------\n'

print(table)
print(mysep)
print(table[table['Borough']=='Not assigned'])

    Postal Code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
..          ...               ...   
160         M8X         Etobicoke   
165         M4Y  Downtown Toronto   
168         M7Y      East Toronto   
169         M8Y         Etobicoke   
178         M8Z         Etobicoke   

                                          Neighborhood  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
5                     Lawrence Manor, Lawrence Heights  
6          Queen's Park, Ontario Provincial Government  
..                                                 ...  
160      The Kingsway, Montgomery Road, Old Mill North  
165                               Church and Wellesley  
168              Business reply mail Proce

In [61]:
table = table.reset_index(drop=True)
table

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


A snapshot of dataframe statistics

In [13]:
table.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M1V,North York,Downsview
freq,1,24,4


In [16]:
table[table['Postal Code']=='M5A']

,Postal Code,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [17]:
table.shape

(103, 3)

## Use persistent requests to obtain geocoordinates for Postal Codes, then append to dataframe.

*Geocoder didn't work*

In [31]:
# initialize your variable to None
lat_lng_coords = None
postal_code='M5G'

# # loop until you get the coordinates
# i=0
# while(lat_lng_coords is None):
#     g = geocoder.google(f'{postal_code}, Toronto, Ontario')
#     print(i)
#     lat_lng_coords = g.latlng
#     i+=1

g = geocoder.google('M5G, Toronto, Ontario')
lat_lng_coords = g.latlng
    
print(g)

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

<[REQUEST_DENIED] Google - Geocode [empty]>


*Try Nominatim*

In [42]:
from geopy.geocoders import Nominatim
geolocator = Nominatim('ca',user_agent="myapp1")
postal_code='M5A'
location = geolocator.geocode({"postalcode": postal_code,'countryRegion': 'CA'})
print(location)
# print((location.latitude, location.longitude))

None


*Try pgeocode*

In [49]:
import pgeocode
nomi = pgeocode.Nominatim('ca')
postal_code='M5A'
location = nomi.query_postal_code(postal_code)
print(location.latitude, location.longitude)

43.6555 -79.3626


Loop through and add lat and long to dataframe

In [67]:
for index, row in table.iterrows():
#     print(table.at[index,'Postal Code'])
    location = nomi.query_postal_code(table.at[index,'Postal Code'])
#     print(location.latitude, location.longitude)
    table.at[index,'latitude'] = location.latitude
    table.at[index,'longitude'] = location.longitude

In [68]:
table

,Postal Code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,Business reply mail Processing Centre,43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939
